In [573]:
# Initializing display and earthengine
from IPython.display import Image
%matplotlib inline

import ee
ee.Initialize()

# importing ipygee for dynamic mapping
from ipygee import *
# import geetools for cloud /sadow discrimination
from geetools import ui, cloud_mask

Map = Map() # from ipygee
Map.show()

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [574]:
import datetime
from matplotlib import dates
import matplotlib.dates as mdates
from pylab import *

In [575]:
# adding landsat8 layer
l8sr = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
# l8sr = ee.ImageCollection("LANDSAT/LE07/C01/T1")


In [576]:
limitrophe= ee.FeatureCollection("users/mahmuduraveek/limitrophe_up");
Map.addLayer(limitrophe, {'color': 'red', 'fillColor': '00000000', 'outline': 1}, 'limitrophe');
Map.centerObject(limitrophe);

In [577]:
# function of clipping image is not working
# def im_Clipping(image):
#     clipped=image.clip(roibuffer)
#     return clipped

# l8sr_clipped=l8sr.map(im_Clipping)

# trueColor = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.5}

# Map.addLayer(l8sr_clipped, trueColor, name='ClippedImage');

In [578]:
l8clip = l8sr.mean().clip(limitrophe); # or you can use median (i think)
Map.addLayer(l8clip,{},'clip');

In [579]:
spatialFiltered = l8sr.filterBounds(limitrophe)
temporalFiltered = spatialFiltered.filterDate('2014-10-01', '2014-10-30')
sortedA = temporalFiltered.sort('CLOUD_COVER')

scene = ee.Image(sortedA.first())
#scene = scene.multiply(0.0001)

#Map.centerObject(scene, 9);

#Map.addLayer(scene, {}, 'default RGB');
#vegPalette= ['#0048FB','#FFFFFF','#CE7E45','#FAD163','#74A909','#3A7405','#1A3B03']
#visParams ={'bands': ['B4', 'B3', 'B2'], 'min': -.2, 'max': 1}
#visParams={min: -0.2, max: 1, 'palette': vegPalette}
#Map.addLayer(scene, visParams, 'true-color composite');

Calculating the NDVI

In [580]:
# nir = scene.select('B5');
# red = scene.select('B4');
# ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI');
# Map.centerObject(scene, 9);
# ndviParams = {'min': 0,'max': 1, 'palette': ['#0000FF', '#FF0000', '#008000']};
# Map.addLayer(ndvi, ndviParams, 'NDVI image');
bands=['B6','B5', 'B4','B2']
ndvi02 = scene.normalizedDifference(['B5', 'B4']).rename('NDVI');
ndvi02= ndvi02.clip(limitrophe);

vegPalette= ['#0048FB','#FFFFFF','#CE7E45','#FAD163','#74A909','#3A7405','#1A3B03']
# vegPalette = ['white','yellow', 'green','blue']
Map.addLayer(ndvi02, {min: -0.2, max: 1, 'palette': vegPalette},name = 'NDVI')

psum=ndvi02.reduceRegion(ee.Reducer.sum(), limitrophe.geometry(),30)
print(psum.getInfo())
#Map.addLayer(ndvi02, ndviParams, 'NDVI image 1');

{'NDVI': 6520.364073878494}


In [581]:
# an effort to exclude the encroached farm lands;
# pixs = ndvi02.lte(0.05).multiply(ee.Image.pixelArea());
# print(pixs.getInfo());


In [582]:
# psum=pixs.reduceRegion(ee.Reducer.sum(), limitrophe.geometry(),30);
# Map.centerObject(limitrophe);
# print(psum.getInfo())

In [583]:
chart_ts = chart.Image.series(**{
    'imageCollection': temporalFiltered, 
    'region': limitrophe,
    'scale': 30,
    'bands': bands,
    'label_bands':['B6','B5', 'B4','B2'],
    'properties':['CLOUD_COVER'],
    #'label_properties':['CLOUD_COVER']
})

In [584]:
chart_ts.dataframe

,B6,B5,B4,B2,CLOUD_COVER
2014-10-02 18:10:21.860,3436.790914,3162.093184,2083.485607,1159.432786,0.00
2014-10-02 18:10:45.750,3473.808681,3176.792883,2104.816786,1166.808140,0.03
2014-10-09 18:16:34.200,3125.847201,2854.704582,1836.306883,1006.065033,0.04
2014-10-18 18:10:25.750,3169.183924,2934.040912,1880.719536,1043.654265,16.45
2014-10-18 18:10:49.650,3207.853671,2949.422280,1905.049997,1055.588620,0.75
2014-10-25 18:16:34.750,3177.932139,2876.306306,1882.066971,1037.287827,9.33


In [585]:
chart_ts.renderWidget(width='50%')

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [586]:
# how to export as a csv?
# Export.table.toDrive({
#   collection: chart_ts.dataframe,
#   description:'vectorsToDriveExample',
#   fileFormat: 'CSV'
# });

In [587]:
# // Make a feature without geometry and set the properties to the dictionary of means.
feature = ee.Feature(limitrophe, bands)

# // Wrap the Feature in a FeatureCollection for export.
featureCollection = ee.FeatureCollection([feature])

# // Export the FeatureCollection.
task = ee.batch.Export.table.toDrive(
  collection=featureCollection,
  description= 'exportTableExample2',
  folder="GEEexport",
  fileFormat='CSV'
)

task.start()

In [588]:
# regions =ee.FeatureCollection("users/mahmuduraveek/limitrophe")
# l8sr = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").filterDate('2013-03-01', '2018-10-30').select(['B5','B4'])
# tempTimeSeries = ui.Chart.image.seriesByRegion(l8sr, regions, ee.Reducer.mean(), 'B5', 200, 'system:time_start', 'label')
#          .setChartType('ScatterChart')
#          .setOptions({
#          title: 'NIR over time in regions of the American West',
#          vAxis: {title: 'Temperature (Kelvin)'},
#          lineWidth: 1,
#          pointSize: 4});


In [589]:
# task = ee.batch.Export.table.toDrive(
#   collection=chart_ts,
#   description= 'exportTableExample',
#   folder="GEEexport",
#   fileFormat='CSV'
# )

# task.start()

#### Doing it for the -NDBSI.
The Normalized Difference Bare Soil Index (NDBSI) aids in the differentiation of no vegetation locations:

NDBI = ((SWIR + RED) - (NIR + BLUE)) / ((SWIR + RED) + (NIR + BLUE))

In [590]:
b6b4 = scene.select('B6').add(scene.select('B4'))
b5b2 = scene.select('B5').add(scene.select('B2'))
NDBSI = b5b2.subtract(b6b4).divide(b5b2.add(b6b4)) #normalized difference bare soil index
NDBSI2= NDBSI.clip(limitrophe);
# barePalette = waterPalette = [ 'blue','white']

#Map.addLayer(NDBSI, {'min': -1, 'max': 1, 'palette': vegPalette}, 'NDBSI')
Map.addLayer(NDBSI2, {'min': -1, 'max': 1, 'palette': vegPalette}, 'NDBSI2')


# filtering only bare soil
bareMask= NDBSI2.lte(0)
NDBSIMasked=NDBSI2.updateMask(bareMask)
Map.addLayer(NDBSIMasked, {'min': -1, 'max': 1, 'palette': vegPalette}, 'NDBSI masked')

In [591]:
# bands=['B6','B5', 'B4','B2']
# NDBSI2 = scene.normalizedDifference(['B5', 'B4']).rename('NDVI');
# ndvi02= NDBSI.clip(limitrophe);

# # vegPalette= ['#0048FB','#FFFFFF','#CE7E45','#FAD163','#74A909','#3A7405','#1A3B03']
# # # vegPalette = ['white','yellow', 'green','blue']
# # Map.addLayer(ndvi02, {min: -0.2, max: 1, 'palette': vegPalette},name = 'NDVI')

# # psum=ndvi02.reduceRegion(ee.Reducer.sum(), limitrophe.geometry(),30)
# # print(psum.getInfo())
# # #Map.addLayer(ndvi02, ndviParams, 'NDVI image 1');

In [592]:
evi = scene.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR': scene.select('B5'),
      'RED': scene.select('B4'),
      'BLUE': scene.select('B2')
})
evi= evi.clip(limitrophe);
# barePalette = waterPalette = [ 'blue','white']

Map.addLayer(evi, {min: -0.2, max: 1, 'palette': vegPalette}, 'EVI')